In [1]:
#Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import json
import os

print("TensorFlow Version:", tf.__version__)


TensorFlow Version: 2.19.0


In [2]:
#Load IMDb Dataset
vocab_size = 20000
maxlen = 200

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=vocab_size)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test  = pad_sequences(x_test, maxlen=maxlen)

print("Training:", x_train.shape)
print("Testing:", x_test.shape)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training: (25000, 200)
Testing: (25000, 200)


In [3]:
#Build LSTM(Long Short-Term Memory)
def build_lstm():
    model = keras.Sequential([
        layers.Embedding(vocab_size, 128, input_length=maxlen),
        layers.SpatialDropout1D(0.2),
        layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

model = build_lstm()
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
#Compile & Train
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.1
)


Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 136s 750ms/step - accuracy: 0.6593 - loss: 0.5967 - val_accuracy: 0.8572 - val_loss: 0.3459
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 130s 741ms/step - accuracy: 0.8740 - loss: 0.3209 - val_accuracy: 0.8560 - val_loss: 0.3377
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 134s 761ms/step - accuracy: 0.9017 - loss: 0.2642 - val_accuracy: 0.8476 - val_loss: 0.3634
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 150s 807ms/step - accuracy: 0.9233 - loss: 0.2161 - val_accuracy: 0.8548 - val_loss: 0.3590
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 142s 807ms/step - accuracy: 0.9438 - loss: 0.1599 - val_accuracy: 0.8452 - val_loss: 0.4347


In [5]:
#Test Accuracy Output
loss, acc = model.evaluate(x_test, y_test)
print(f"\n🟢 FINAL TEST ACCURACY: {acc:.4f}")


782/782 ━━━━━━━━━━━━━━━━━━━━ 46s 59ms/step - accuracy: 0.8358 - loss: 0.4588

🟢 FINAL TEST ACCURACY: 0.8398


In [6]:
#Sample Predictions
pred = model.predict(x_test[:5])

print("Predicted Sentiment Scores:", pred.flatten())
print("Actual Labels:", y_test[:5])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
Predicted Sentiment Scores: [0.12884729 0.9958587  0.9743671  0.74813056 0.9994658 ]
Actual Labels: [0 1 1 0 1]
